In [ ]:
import pandas as pd
import numpy as np

In [ ]:
REMOVER_path = "../data/raw/REMOVER.xlsx"

In [ ]:
df_REMOVER = pd.read_excel(REMOVER_path, sheet_name=0)

df_REMOVER

In [ ]:
# df_REMOVER já está carregado
colmap = {
    'LATITUDE (°) - tide free': 'lat',
    'LONGITUDE (°) - tide free': 'lon',
    'H (m) - zero tide': 'H_zerotide_m',
    'Anomalia de gravidade de Helmert (mGal) - zero tide': 'dg_helmert_mgal',
    'Anomalia de gravidade GOCO06s até 300 (mGal)': 'dg_goco_mgal',
    'Efeito indireto em Anomalia de gravidade de Helmert (mGal)': 'dg_indirect_mgal',
    'Anomalia de gravidade residual (mGal)': 'dg_residual_planilha_mgal',
}
df = df_REMOVER.rename(columns=colmap).copy()


# ordenar colunas de interesse
df = df[['lat','lon','H_zerotide_m','dg_helmert_mgal','dg_goco_mgal','dg_indirect_mgal','dg_residual_planilha_mgal']]
len(df), df.head(3)

### Recalcular a anomalia residual e checar contra a planilha

Fórmula:

$$
\Delta g_{\text{res}}=\Delta g_{\text{Helmert}}-\Delta g_{\text{GOCO06s}}-\delta_{\text{indireto}}
$$

In [ ]:
df['dg_res_calc_mgal'] = (
    df['dg_helmert_mgal']
    - df['dg_goco_mgal']
    - df['dg_indirect_mgal']
)

df['erro_mgal'] = df['dg_res_calc_mgal'] - df['dg_residual_planilha_mgal']

print(df[['dg_res_calc_mgal','dg_residual_planilha_mgal','erro_mgal']].head())
print(
    f"Erro abs. máx = {df['erro_mgal'].abs().max():.6f} mGal | "
    f"RMSE = {np.sqrt((df['erro_mgal']**2).mean()):.6f} mGal"
)


In [ ]:
# tirar linhas com NaN nas colunas essenciais
keep = ['lat','lon','dg_res_calc_mgal']
clean = df.dropna(subset=keep).copy()

# salvar um CSV "canônico" para a próxima etapa
clean.to_csv("../data/processed/remove_pontos.csv", index=False)

# opcional: também salvar um TXT sem cabeçalho (se quiser reproduzir formato de scripts antigos)
clean[['lat','lon','dg_res_calc_mgal']].to_csv(
    "../data/processed/remove_pontos.txt",
    index=False, header=False, float_format="%.6f"
)

len(clean), clean.head(3)


In [ ]:
import matplotlib.pyplot as plt

plt.hist(clean['dg_res_calc_mgal'], bins=20)
plt.title("Anomalia de gravidade residual (mGal)")
plt.xlabel("mGal"); plt.ylabel("Contagem")
plt.show()

plt.scatter(clean['lon'], clean['lat'], c=clean['dg_res_calc_mgal'])
plt.title("Pontos e Δg_res (mGal)")
plt.xlabel("lon"); plt.ylabel("lat")
plt.colorbar(label="mGal")
plt.show()
